In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
%matplotlib inline

In [2]:
trainSet  = pd.read_csv('train.csv')
trainingEncoded = pd.get_dummies(trainSet)
x = trainingEncoded.drop(['hand'], axis=1)
y = trainingEncoded['hand']
xTrain, xVal, yTrain, yVal = train_test_split(x, 
                                              y,
                                              test_size=.1,
                                              random_state=12)

In [3]:
model = RandomForestClassifier(n_estimators=30, random_state=12)

In [4]:
clf = Pipeline([
    ('feature_selection', RFE(model, 5)),
    ('classification', RandomForestClassifier(n_estimators=30))
])
clf.fit(xTrain, yTrain)

Pipeline(memory=None,
     steps=[('feature_selection', RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_spl...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [5]:
pipePredicted = clf.predict(xVal)

pdPipePredicted = pd.DataFrame(data=pipePredicted, columns=['hand'])
print pdPipePredicted.hand.value_counts()

print metrics.classification_report(yVal, pdPipePredicted)

0    1430
1    1033
2      26
3      10
4       2
Name: hand, dtype: int64
             precision    recall  f1-score   support

          0       0.70      0.82      0.75      1220
          1       0.64      0.61      0.62      1088
          2       0.54      0.11      0.18       126
          3       0.60      0.11      0.18        55
          4       0.50      0.17      0.25         6
          5       0.00      0.00      0.00         6

avg / total       0.66      0.67      0.65      2501



/Users/kristiansuhartono/python-env/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
